# Carga da Dimensão Ativo Financeiro

Este notebook realiza a carga da dimensão ativo financeiro (dim_ativo_financeiro) a partir dos dados da tabela staging de cotação histórica.

## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_timestamp, max, count, sha2, concat, coalesce, when
from delta.tables import DeltaTable

## Start Spark Session

In [ ]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Carga Dimensão Ativo Financeiro") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Define Delta Table Paths

In [3]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"

## Read Source Data

In [4]:
# Lê a tabela staging de cotação histórica
df_stg_cotacao_historica = spark.read.format("delta").load(delta_path_cotacao_historica)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_cotacao_historica}")
print(f"Total de registros: {df_stg_cotacao_historica.count()}")

[SUCESSO] Leitura da tabela staging em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica
Total de registros: 2396811


## Transform Data

In [5]:
# Filtra apenas registros com tp_mercado = '10'
df_stg_cotacao_historica = df_stg_cotacao_historica.filter(col("tp_mercado") == "010")

# Faz o agrupamento dos dados e busca a maior especificação do papel
df_grouped = df_stg_cotacao_historica.groupBy(
    "cd_negociacao",
    "tp_mercado",
    "nm_empresa"
).agg(
    max(col("ds_especificacao_papel")).alias("ds_especificacao_papel"),
    count("*").alias("count")
).orderBy("cd_negociacao")

# Gera chave surrogate e transforma os dados
df_dim_ativo = df_grouped.select(
    # Gera surrogate key usando sha2 do cd_negociacao concatenado com nm_empresa
    sha2(concat(col("cd_negociacao"), col("nm_empresa")), 256).alias("sk_ativo"),
    col("cd_negociacao").alias("cd_ativo"),
    col("nm_empresa"),
    col("ds_especificacao_papel").alias("ds_ativo"),
    col("tp_mercado").alias("cd_tipo_mercado"),
    when(col("tp_mercado") == "010", "VISTA")
    .when(col("tp_mercado") == "012", "EXERCÍCIO DE OPÇÕES DE COMPRA")
    .when(col("tp_mercado") == "013", "EXERCÍCIO DE OPÇÕES DE VENDA")
    .when(col("tp_mercado") == "017", "LEILÃO")
    .when(col("tp_mercado") == "020", "FRACIONÁRIO")
    .when(col("tp_mercado") == "030", "TERMO")
    .when(col("tp_mercado") == "050", "FUTURO COM RETENÇÃO DE GANHO")
    .when(col("tp_mercado") == "060", "FUTURO COM MOVIMENTAÇÃO CONTÍNUA")
    .when(col("tp_mercado") == "070", "OPÇÕES DE COMPRA")
    .when(col("tp_mercado") == "080", "OPÇÕES DE VENDA")
    .otherwise("DESCONHECIDO")
    .alias("ds_tipo_mercado"),
    lit(None).cast("string").alias("cd_isin"),
    lit("AÇÃO").alias("ds_tipo_ativo")
)

## Write Data

In [6]:
# Carrega a tabela Delta existente
deltaTable = DeltaTable.forPath(spark, delta_path_dim_ativo)

# Realiza o merge (upsert) na tabela Delta
(
    deltaTable.alias("target")
    .merge(
        df_dim_ativo.alias("source"),
        "target.sk_ativo = source.sk_ativo"
    )
    # Quando o registro já existe e há diferença nos campos relevantes → atualiza
    .whenMatchedUpdate(
        condition="""
            coalesce(target.ds_ativo, '') != coalesce(source.ds_ativo, '') OR 
            coalesce(target.cd_tipo_mercado, '') != coalesce(source.cd_tipo_mercado, '') OR 
            coalesce(target.ds_tipo_mercado, '') != coalesce(source.ds_tipo_mercado, '') OR 
            coalesce(target.cd_isin, '') != coalesce(source.cd_isin, '') OR
            coalesce(target.ds_tipo_ativo, '') != coalesce(source.ds_tipo_ativo, '')
        """,
        set={
            "ds_ativo": "source.ds_ativo",
            "cd_tipo_mercado": "source.cd_tipo_mercado",
            "ds_tipo_mercado": "source.ds_tipo_mercado",
            "cd_isin": "source.cd_isin",
            "ds_tipo_ativo": "source.ds_tipo_ativo",
            "ts_atualizacao": "current_timestamp()"  # Atualiza timestamp no update
        }
    )
    # Quando o registro não existe → insere novo
    .whenNotMatchedInsert(values={
        "sk_ativo": "source.sk_ativo",
        "cd_ativo": "source.cd_ativo",
        "nm_empresa": "source.nm_empresa",
        "ds_ativo": "source.ds_ativo",
        "cd_tipo_mercado": "source.cd_tipo_mercado",
        "ds_tipo_mercado": "source.ds_tipo_mercado",
        "cd_isin": "source.cd_isin",
        "ds_tipo_ativo": "source.ds_tipo_ativo",
        "ts_insercao": "current_timestamp()",  # Data/hora da inserção
        "ts_atualizacao": "null"               # Ainda não foi atualizado
    })
    .execute()
)

# 📊 Obtém métricas do último merge
last_operation = deltaTable.history(1).select("operationMetrics").collect()[0][0]
inserted = int(last_operation.get("numTargetRowsInserted", 0))
updated = int(last_operation.get("numTargetRowsUpdated", 0))

print(f"✅ Merge realizado com sucesso")
print(f"🆕 Inseridos: {inserted}")
print(f"♻️ Atualizados: {updated}")

✅ Merge realizado com sucesso
🆕 Inseridos: 0
♻️ Atualizados: 0


## Stop Spark Session

In [7]:
# Encerra a SparkSession
spark.stop()